In [ ]:
import pandas as pd
import plotly.express as px

# See: https://towardsdatascience.com/read-data-from-google-sheets-into-pandas-without-the-google-sheets-api-5c468536550
sheet_id = "1q8MB-H49pd1ojqZzMbs0ExbMkWKl0Ll0cyJ3_OpF3_g"
sheet_name = "Job_Search_Status"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url, usecols=range(0, 11))

In [ ]:
df

,Company,Industry,Position,Application Link,Location,Status,Source,App Date,My TODO,Vivian's TODO,Notes
0,Fakeaccepted,NaN,Fake,NaN,Fake,Accepted,Fake,4/15/2022,NaN,NaN,Fake
1,Fakechosen,NaN,Fake,NaN,Fake,Chosen,Fake,4/15/2022,NaN,NaN,Fake
2,Fakechosen2,NaN,Fake,NaN,Fake,Chosen,Fake,4/15/2022,NaN,NaN,Fake
3,Fakeonsite,NaN,Fake,NaN,Fake,Onsite,Fake,4/15/2022,NaN,NaN,Fake
4,Fakeonsite2,NaN,Fake,NaN,Fake,Onsite,Fake,4/15/2022,NaN,NaN,Fake
...,...,...,...,...,...,...,...,...,...,...,...
129,Guided.gg,Gaming,Data Scientist,https://jobs.lever.co/guilded/3a85f86d-f141-41...,Remote,Applied,LinkedIn,5/2/2022,NaN,NaN,hn
130,Faire,NaN,Data Scientist New Graduate,https://boards.greenhouse.io/faire/jobs/607276...,Remote,Applied,LinkedIn,5/2/2022,NaN,NaN,hn
131,Tonal,Gym,Data Scientist,https://jobs.lever.co/tonal/af574c9a-9a36-41af...,NYC,Applied,LinkedIn,5/2/2022,NaN,NaN,hn
132,Garner Health,Healthcare,Data Science Engineer,https://boards.greenhouse.io/garnerhealth/jobs...,NYC,Interview,NYC DSA,5/2/2022,Interview scheduled with Doug,NaN,NaN


In [ ]:
nycdsa = (
    df.groupby("Source")["Company"]
    .nunique()
    .reset_index()
    .rename(columns={"Company": "Positions"})
)
fig = px.pie(nycdsa, values="Positions", names="Source", title="Jobs via Source")
colors = ["gold", "mediumturquoise", "darkorange", "lightgreen"]
fig.update_traces(
    hoverinfo="label+percent",
    textinfo="value",
    textfont_size=20,
    marker=dict(colors=colors, line=dict(color="#151515", width=2)),
)
fig.show()

In [ ]:
industry = (
    df.groupby("Industry")["Company"]
    .nunique()
    .reset_index()
    .rename(columns={"Company": "Jobs"})
    .sort_values(by="Jobs", ascending=False)
    .head(10)
)

fig = px.bar(industry, title="Jobs by Industry (top 10)", x="Industry", y="Jobs")
fig.show()